# Overview

This file is used to:

- Create numpy versions of the series embedding file (convert from .h5 to numpy) for both the training and test set; easily loaded in when needed during execution
- Generate embeddings for the domain information and calculate PCA on those embeddings
- Calculate the information accretion (IA)

In [ ]:
# Can be used if running in a google colab notebook
# from google.colab import drive

# drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# Specify the path to the project folder (biological_data_pfp); must have '/' at the end
# PATH = '/content/drive/MyDrive/UniPD/BD/Biological Data Project/biological_data_pfp/'
PATH = 'biological_data_pfp/'

TRAIN_SET_TSV_PATH = PATH + 'train/train_set.tsv'
EMBEDDINGS_H5_PATH = PATH + 'train/train_embeddings.h5'
EMBEDDINGS_H5_PATH_TEST = PATH + 'test/test_embeddings.h5'
NPY_FILE_PATH_EMBEDDINGS = PATH + 'train/train_embeddings.npy'
NPY_FILE_PATH_EMBEDDINGS_TEST = PATH + 'test/test_embeddings.npy'
NPY_FILE_PATH_IDS = PATH + 'train/train_ids.npy'
NPY_FILE_PATH_IDS_TEST = PATH + 'test/test_ids.npy'
PROTEIN2IPR_FILE_PATH = PATH + 'train/train_protein2ipr.dat'
PROTEIN2IPR_FILE_PATH_TEST = PATH + 'test/test_protein2ipr.dat'
DOMAIN_EMBEDDINGS_PCA_IDS_FILE_PATH = PATH + 'train/domain_embeddings_pca_ids.npy'
DOMAIN_EMBEDDINGS_PCA_IDS_FILE_PATH_TEST = PATH + 'test/test_domain_embeddings_pca_ids.npy'
NPY_PCA_DOMAIN_FILE_PATH = PATH + 'train/domain_embeddings_pca.npy'
NPY_PCA_DOMAIN_FILE_PATH_TEST = PATH + 'test/test_domain_embeddings_pca.npy'
TRAIN_SET_IA_TSV_FILE_PATH = PATH + 'train/train_set_ia.tsv'
GO_BASIC_OBO_FILE_PATH = PATH + 'train/go-basic.obo'


In [ ]:
# Imports
import h5py
import numpy as np
import pandas as pd
import shutil
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

# Train/Val Split

This creates the train and val sets and saves them to file for use in training all the models.

In [ ]:
# Load the training dataset and split the data into train and validation sets

train_set_df = pd.read_csv(PATH + 'train/train_set.tsv', sep='\t')
X = train_set_df.Protein_ID.unique().tolist()

X_train, X_val, _,_ = train_test_split(X, X, train_size=0.8, random_state=42)

train_set_df.loc[train_set_df.Protein_ID.isin(X_train)].to_csv(PATH + 'train/train.tsv',
                                                        sep='\t', index=False)
train_set_df.loc[train_set_df.Protein_ID.isin(X_val)].to_csv(PATH + 'train/validation.tsv',
                                                        sep='\t', index=False)

len(X_train), len(X_val)

(99175, 24794)

# Convert the Sequence Embeddings to npy

## Train Set

In [ ]:
# Load the sequence training embeddings
emb = h5py.File(EMBEDDINGS_H5_PATH, 'r')
keys = list(emb.keys())

# Create a numpy array for embeddings
X = np.zeros((len(keys), len(emb[keys[0]])))

# Create a numpy array for IDs
ids = np.array(keys)

# Fill the embeddings array
for i, k in enumerate(keys):
    X[i, :] = np.array(emb[k])

# Save embeddings to a numpy file
with open(NPY_FILE_PATH_EMBEDDINGS, 'wb') as f:
    np.save(f, X)

# Save IDs to a separate numpy file
with open(NPY_FILE_PATH_IDS, 'wb') as f:
    np.save(f, ids)

## Test Set

In [ ]:
# Load the sequence test embeddings
emb_test = h5py.File(EMBEDDINGS_H5_PATH_TEST, 'r')
keys_test = list(emb_test.keys())

# Create a numpy array for test embeddings
X_test = np.zeros((len(keys_test), len(emb_test[keys_test[0]])))

# Create a numpy array for test IDs
ids_test = np.array(keys_test)

# Fill the test embeddings array
for i, k in enumerate(keys_test):
    X_test[i, :] = np.array(emb_test[k])

# Save test embeddings to a numpy file
with open(NPY_FILE_PATH_EMBEDDINGS_TEST, 'wb') as f:
    np.save(f, X_test)

# Save test IDs to a separate numpy file
with open(NPY_FILE_PATH_IDS_TEST, 'wb') as f:
    np.save(f, ids_test)

# Generating IPR domains embeddings ids

## Train set

In [ ]:
protein2ipr = pd.read_csv(PROTEIN2IPR_FILE_PATH,header = None, sep='\t')
protein2ipr.columns = ['Protein_ID','IPR_ID','desc','db','start','end']
protein2ipr.head()

,Protein_ID,IPR_ID,desc,db,start,end
0,A0A009IHW8,IPR000157,Toll/interleukin-1 receptor homology (TIR) domain,PF13676,138,231
1,A0A009IHW8,IPR000157,Toll/interleukin-1 receptor homology (TIR) domain,PS50104,133,266
2,A0A009IHW8,IPR000157,Toll/interleukin-1 receptor homology (TIR) domain,SM00255,134,258
3,A0A009IHW8,IPR035897,Toll/interleukin-1 receptor homology (TIR) dom...,G3DSA:3.40.50.10140,80,266
4,A0A009IHW8,IPR035897,Toll/interleukin-1 receptor homology (TIR) dom...,SSF52200,128,249


In [ ]:
protein2ipr

,Protein_ID,IPR_ID,desc,db,start,end
0,A0A009IHW8,IPR000157,Toll/interleukin-1 receptor homology (TIR) domain,PF13676,138,231
1,A0A009IHW8,IPR000157,Toll/interleukin-1 receptor homology (TIR) domain,PS50104,133,266
2,A0A009IHW8,IPR000157,Toll/interleukin-1 receptor homology (TIR) domain,SM00255,134,258
3,A0A009IHW8,IPR035897,Toll/interleukin-1 receptor homology (TIR) dom...,G3DSA:3.40.50.10140,80,266
4,A0A009IHW8,IPR035897,Toll/interleukin-1 receptor homology (TIR) dom...,SSF52200,128,249
...,...,...,...,...,...,...
1103541,X6RM59,IPR006434,"Pyrimidine 5'-nucleotidase, eukaryotic",SFLDG01128,42,331
1103542,X6RM59,IPR006434,"Pyrimidine 5'-nucleotidase, eukaryotic",TIGR01544,50,330
1103543,X6RM59,IPR006434,"Pyrimidine 5'-nucleotidase, eukaryotic",cd07504,59,331
1103544,X6RM59,IPR023214,HAD superfamily,G3DSA:3.40.50.1000,70,327


In [ ]:
unique_protein_ipr_df = protein2ipr[['Protein_ID', 'IPR_ID']].drop_duplicates()
grouped_unique_protein_ipr_df = unique_protein_ipr_df.groupby('Protein_ID')['IPR_ID'].apply(' '.join).reset_index()
print(unique_protein_ipr_df.head())
print(grouped_unique_protein_ipr_df.head())

   Protein_ID     IPR_ID
0  A0A009IHW8  IPR000157
3  A0A009IHW8  IPR035897
5  A0A021WW32  IPR006910
6  A0A021WW32  IPR039781
7  A0A021WW32  IPR049589
   Protein_ID                         IPR_ID
0  A0A009IHW8            IPR000157 IPR035897
1  A0A021WW32  IPR006910 IPR039781 IPR049589
2  A0A021WZA4  IPR004481 IPR004837 IPR044880
3  A0A023FBW7                      IPR045797
4  A0A023FDY8                      IPR045797


In [ ]:
grouped_unique_protein_ipr_df

,Protein_ID,IPR_ID
0,A0A009IHW8,IPR000157 IPR035897
1,A0A021WW32,IPR006910 IPR039781 IPR049589
2,A0A021WZA4,IPR004481 IPR004837 IPR044880
3,A0A023FBW7,IPR045797
4,A0A023FDY8,IPR045797
...,...,...
117442,X6RKQ2,IPR015007
117443,X6RKS3,IPR000817 IPR022416 IPR025860 IPR036924
117444,X6RLP6,IPR000504 IPR012677 IPR034147 IPR035979 IPR045164
117445,X6RLR1,IPR009991


In [ ]:
with open(DOMAIN_EMBEDDINGS_PCA_IDS_FILE_PATH, 'wb') as f:
  np.save(f, np.array(grouped_unique_protein_ipr_df.Protein_ID.values))

In [ ]:
np.array(grouped_unique_protein_ipr_df.Protein_ID.values)

array(['A0A009IHW8', 'A0A021WW32', 'A0A021WZA4', ..., 'X6RLP6', 'X6RLR1',
       'X6RM59'], dtype=object)

## Test set

In [ ]:
protein2ipr_test = pd.read_csv(PROTEIN2IPR_FILE_PATH_TEST,header = None, sep='\t')
protein2ipr_test.columns = ['Protein_ID','IPR_ID','desc','db','start','end']
protein2ipr_test.head()

,Protein_ID,IPR_ID,desc,db,start,end
0,A0A0B4JCV4,IPR007707,Transforming acidic coiled-coil-containing pro...,PF05010,994,1204
1,A0A0B4JCV4,IPR039915,TACC family,PTHR13924,38,1206
2,A0A0B4KHT0,IPR000315,B-box-type zinc finger,PF00643,177,219
3,A0A0B4KHT0,IPR000315,B-box-type zinc finger,PF00643,236,274
4,A0A0B4KHT0,IPR000315,B-box-type zinc finger,PS50119,173,220


In [ ]:
protein2ipr_test

,Protein_ID,IPR_ID,desc,db,start,end
0,A0A0B4JCV4,IPR007707,Transforming acidic coiled-coil-containing pro...,PF05010,994,1204
1,A0A0B4JCV4,IPR039915,TACC family,PTHR13924,38,1206
2,A0A0B4KHT0,IPR000315,B-box-type zinc finger,PF00643,177,219
3,A0A0B4KHT0,IPR000315,B-box-type zinc finger,PF00643,236,274
4,A0A0B4KHT0,IPR000315,B-box-type zinc finger,PS50119,173,220
...,...,...,...,...,...,...
11259,W5U9R6,IPR032397,Rel homology dimerisation domain,PF16179,559,656
11260,W5U9R6,IPR037059,"Rel homology domain (RHD), DNA-binding domain ...",G3DSA:2.60.40.340,375,550
11261,W7K139,IPR002641,Patatin-like phospholipase domain,PF01734,338,544
11262,W7K139,IPR002641,Patatin-like phospholipase domain,PS51635,338,544


In [ ]:
unique_protein_ipr_df_test = protein2ipr_test[['Protein_ID', 'IPR_ID']].drop_duplicates()
grouped_unique_protein_ipr_df_test = unique_protein_ipr_df_test.groupby('Protein_ID')['IPR_ID'].apply(' '.join).reset_index()
print(unique_protein_ipr_df_test.head())
print(grouped_unique_protein_ipr_df_test.head())

    Protein_ID     IPR_ID
0   A0A0B4JCV4  IPR007707
1   A0A0B4JCV4  IPR039915
2   A0A0B4KHT0  IPR000315
8   A0A0B4KHT0  IPR001487
14  A0A0B4KHT0  IPR001841
   Protein_ID                                             IPR_ID
0  A0A0B4JCV4                                IPR007707 IPR039915
1  A0A0B4KHT0  IPR000315 IPR001487 IPR001841 IPR001965 IPR003...
2  A0A0B4P506                                IPR003417 IPR036552
3  A0A0G2K1A2                      IPR010255 IPR019791 IPR037120
4  A0A0G2K1V4  IPR000048 IPR001609 IPR002928 IPR004009 IPR008...


In [ ]:
grouped_unique_protein_ipr_df_test

,Protein_ID,IPR_ID
0,A0A0B4JCV4,IPR007707 IPR039915
1,A0A0B4KHT0,IPR000315 IPR001487 IPR001841 IPR001965 IPR003...
2,A0A0B4P506,IPR003417 IPR036552
3,A0A0G2K1A2,IPR010255 IPR019791 IPR037120
4,A0A0G2K1V4,IPR000048 IPR001609 IPR002928 IPR004009 IPR008...
...,...,...
976,Q9ZP06,IPR001236 IPR001252 IPR001557 IPR010097 IPR015...
977,Q9ZVF3,IPR000916 IPR023393
978,S0HPF7,IPR008707 IPR011047
979,W5U9R6,IPR002909 IPR008366 IPR008967 IPR011539 IPR013...


In [ ]:
with open(DOMAIN_EMBEDDINGS_PCA_IDS_FILE_PATH_TEST, 'wb') as f:
  np.save(f, np.array(grouped_unique_protein_ipr_df_test.Protein_ID.values))

In [ ]:
np.array(grouped_unique_protein_ipr_df_test.Protein_ID.values)

array(['A0A0B4JCV4', 'A0A0B4KHT0', 'A0A0B4P506', 'A0A0G2K1A2',
       'A0A0G2K1V4', 'A0A0K0LT60', 'A0A0K3AUJ9', 'A0A0N7KJT8',
       'A0A0R4ILJ8', 'A0A166U5H3', 'A0A1D8PIH0', 'A0A1D8PJ80',
       'A0A2K8FQU5', 'A0A3Q0KDV9', 'A0A7G6KN55', 'A0A8I5Y222',
       'A0A8I6A5Y4', 'A0A8I6A7Q7', 'A0A8I6AN32', 'A0A8I6AP99',
       'A0A8I6B5D5', 'A1KYB4', 'A1Z6Z8', 'A1Z928', 'A2A124', 'A2A6C4',
       'A2A8U2', 'A2A935', 'A2RU49', 'A2RVQ5', 'A4VAR9', 'A5HTT2',
       'A6HE59', 'A6HHV7', 'A6KTX2', 'A7XY94', 'A8JNK7', 'A8JQY3',
       'A8JRI1', 'A9LNK9', 'B1AWD1', 'B2RXS4', 'B5A5T4', 'B5BM46',
       'B9EJ57', 'C9K505', 'D3ZW27', 'D6XGM6', 'D6XMN0', 'E7FB98',
       'E9Q0S6', 'F1MA19', 'F1MAF1', 'F4HQG6', 'F4HTM3', 'F4ILG6',
       'F4JEW8', 'F4JL28', 'F4JL60', 'F4K6B6', 'F6W2R2', 'F7EMR7',
       'F7J0N2', 'F8VQ29', 'G0XYH3', 'G2TRP5', 'G3V6G7', 'G5E8P1',
       'G5EBF1', 'G5EC23', 'G5EDP9', 'G5EEU2', 'G5EF71', 'G5EFE1',
       'H2L051', 'H2L0Q3', 'I1S9X9', 'J9S2W1', 'M9NE97', 'M9NEI4',
       'M9N

# PCA Domain Embedding Calculation

## Training Set

In [ ]:
# Now, use CountVectorizer to transform the IPR_IDs into a matrix of token counts for the training set
vectorizer = CountVectorizer(token_pattern = r"(?u)\b\w+\b") # This pattern keeps single letter tokens

# Fit and transform to an array of token counts stored as int32 for memory
X = vectorizer.fit_transform(grouped_unique_protein_ipr_df['IPR_ID']).toarray().astype(np.int32)

print(X)

# Now, X is a matrix where each row corresponds to a Protein_ID and each column is an IPR_ID.
# The value in each cell is the count of the IPR_ID for the corresponding Protein_ID.

In [ ]:
# Apply PCA (TruncatedSVD)
n_components = 1024
pca = TruncatedSVD(n_components=n_components)
X_pca = pca.fit_transform(X)

# X_reduced now has shape (117447, 1024)

In [ ]:
# Once the PCA is calculated, delete the token matrix to save on RAM space
del(X)

In [ ]:
# Save the PCA domain embeddings
np.save(NPY_PCA_DOMAIN_FILE_PATH, X_pca)

## Test Set

In [ ]:
# PCA calculation will reduce the number of components to the minimum between the specified # of components and the # of examples in the dataset
# In the case of the test set, there are 981 protein examples, and 1,024 desired components, so the PCA will reduce it to 981 components by default
# To obtain the desired shape of the domain embeddings for th emodels (must match the 1,024 for the sequence embeddings), 43 randomly
#    sampled proteins are duplicated onto the end of the domain_embeddings dataframe for the pcs calculation and removed after.

# Duplicate 43 random proteins to add on the end of the test set to make it the same size as the training set
grouped_unique_protein_ipr_df_test = pd.concat([grouped_unique_protein_ipr_df_test, grouped_unique_protein_ipr_df_test.sample(n=43, random_state=42)])
print(grouped_unique_protein_ipr_df_test.shape)

(1024, 2)


In [ ]:
# Now, use CountVectorizer to transform the IPR_IDs into a matrix of token counts for the test set
vectorizer_test = CountVectorizer(token_pattern = r"(?u)\b\w+\b") # This pattern keeps single letter tokens

# Fit and transform to an array of token counts stored as int32 for memory
X_test = vectorizer_test.fit_transform(grouped_unique_protein_ipr_df_test['IPR_ID']).toarray().astype(np.int32)

print(X_test)

# Now, X_test is a matrix where each row corresponds to a Protein_ID and each column is an IPR_ID.
# The value in each cell is the count of the IPR_ID for the corresponding Protein_ID.

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
# Apply PCA (TruncatedSVD) to the test set
n_components_test = 1024
pca_test = TruncatedSVD(n_components=n_components_test)
X_pca_test = pca_test.fit_transform(X_test)

# X_reduced now has shape (117447, 1024)

In [ ]:
X_pca_test.shape

(1024, 1024)

In [ ]:
# Again, delete the matrix to save RAM
del(X_test)

In [ ]:
# drop the last 43 rows of the test set to make it the original size of the test set
X_pca_test = X_pca_test[:-43]

print(X_pca_test.shape)

(981, 1024)


In [ ]:
# Save the PCA-transformed domains for the test set
np.save(NPY_PCA_DOMAIN_FILE_PATH_TEST, X_pca_test)

# IA calculation

In [ ]:
# Clone the InformationAccretion repository
!git clone https://github.com/claradepaolis/InformationAccretion.git

# Change directory to the InformationAccretion folder
%cd InformationAccretion


c:\Users\camer\OneDrive\Documents\CMK\College\Padova\Fall2023\BiologicalData\BioDataFinalGroup8\InformationAccretion


Cloning into 'InformationAccretion'...


In [ ]:
train_set_ia = pd.read_csv('../' + TRAIN_SET_TSV_PATH, sep='\t')

train_set_ia.columns = ["EntryID", "aspect", "term"]

new_order = ["EntryID", "term", "aspect"]
train_set_ia = train_set_ia[new_order]

aspect_mapping = {
        'biological_process': 'BPO',
        'molecular_function': 'MFO',
        'cellular_component': 'CCO'
    }

    # Map the values in the 'aspect' column using the dictionary
train_set_ia['aspect'] = train_set_ia['aspect'].map(aspect_mapping)
train_set_ia

,EntryID,term,aspect
0,P91124,GO:0005575,CCO
1,P91124,GO:0110165,CCO
2,P91124,GO:0005737,CCO
3,P91124,GO:0005622,CCO
4,P91124,GO:0043226,CCO
...,...,...,...
4277042,P28271,GO:0010608,BPO
4277043,P28271,GO:0080090,BPO
4277044,P28271,GO:0006417,BPO
4277045,P28271,GO:0051246,BPO


In [ ]:
train_set_ia.to_csv('../' + TRAIN_SET_IA_TSV_FILE_PATH,sep='\t', index=False)

In [ ]:
pd.read_csv('../' + TRAIN_SET_IA_TSV_FILE_PATH, sep='\t')

,EntryID,term,aspect
0,P91124,GO:0005575,CCO
1,P91124,GO:0110165,CCO
2,P91124,GO:0005737,CCO
3,P91124,GO:0005622,CCO
4,P91124,GO:0043226,CCO
...,...,...,...
4277042,P28271,GO:0010608,BPO
4277043,P28271,GO:0080090,BPO
4277044,P28271,GO:0006417,BPO
4277045,P28271,GO:0051246,BPO


In [ ]:

# Install obonet, necessary to calculate the IA
!pip install obonet


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\camer\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
# Move the go basic obo file to the current directory for ease in execution
source_path = '../' + GO_BASIC_OBO_FILE_PATH

destination_directory = "go-basic.obo"
shutil.copy(source_path, destination_directory)

# move the train_set_ia.tsv to the current directory for ease un execution
source_path = '../' + TRAIN_SET_IA_TSV_FILE_PATH

destination_directory = "train_set_ia.tsv"
shutil.copy(source_path, destination_directory)

'train_set_ia.tsv'

In [ ]:
!python ia.py --annot train_set_ia.tsv --graph go-basic.obo --outfile IA.txt


Counting Terms
Computing Information Accretion
Saving to file IA.txt


In [ ]:
ia_df = pd.read_csv('IA.txt', sep='\t', header=None)
ia_df

,0,1
0,GO:0000001,0.000000
1,GO:0000002,10.147205
2,GO:0000003,3.402925
3,GO:0000011,0.000000
4,GO:0000012,10.445015
...,...,...
42832,GO:2001083,7.139551
42833,GO:2001084,7.629357
42834,GO:2001085,7.139551
42835,GO:2001147,7.000000


In [ ]:
# Move the IA file back into the biological_data_pfp folder
source_path = "IA.txt"

destination_directory = '../' + PATH + "train/IA.txt"
shutil.copy(source_path, destination_directory)

'../biological_data_pfp/train/IA.txt'